In [1]:
import pandas as pd
import numpy as np
import ast

### Extract necessary information from the data

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [ ]:
df = spark.read.format("json").load("foundationDownload.json")

In [ ]:
df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- dataType: string (nullable = true)
 |-- description: string (nullable = true)
 |-- fdcId: long (nullable = true)
 |-- foodAttributes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- foodCategory: struct (nullable = true)
 |    |-- description: string (nullable = true)
 |-- foodClass: string (nullable = true)
 |-- foodNutrients: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- amount: double (nullable = true)
 |    |    |-- dataPoints: long (nullable = true)
 |    |    |-- foodNutrientDerivation: struct (nullable = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- foodNutrientSource: struct (nullable = true)
 |    |    |    |    |-- code: string (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- id: long (nullable = true)
 |   

In [ ]:
# Select the required fields from the DataFrame
df_selected = df.select(
    "description",
    "foodNutrients.amount",
    "foodNutrients.nutrient.name",
    "foodNutrients.nutrient.unitName",
    "foodPortions.amount",
    "foodPortions.gramWeight",
    "foodPortions.measureUnit.name"

)

# Show the results
df_selected.show(truncate=False)


+-----------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.sql import functions as F

# Explode the foodNutrients.nutrient.name array to get one nutrient name per row
exploded_df = df.select(F.explode("foodNutrients.nutrient.name").alias("nutrient_name"))

# Count total number of rows in the original DataFrame
total_rows = df.count()

# Find nutrient names that appear in every row
common_nutrients = (
    exploded_df.groupBy("nutrient_name")
    .count()
    .filter(F.col("count") > 30/100 *total_rows)
    .select("nutrient_name")
)

# Show the common nutrients
common_nutrients.show(40, truncate=False)


+----------------------------------+
|nutrient_name                     |
+----------------------------------+
|Magnesium, Mg                     |
|Water                             |
|Thiamin                           |
|Manganese, Mn                     |
|Calcium, Ca                       |
|Energy                            |
|Selenium, Se                      |
|Zinc, Zn                          |
|Energy (Atwater Specific Factors) |
|Vitamin B-6                       |
|Fiber, total dietary              |
|Fructose                          |
|Sugars, Total                     |
|Riboflavin                        |
|Fatty acids, total saturated      |
|Sodium, Na                        |
|Phosphorus, P                     |
|Energy (Atwater General Factors)  |
|Fatty acids, total monounsaturated|
|Total lipid (fat)                 |
|Carbohydrate, by difference       |
|Potassium, K                      |
|Ash                               |
|Protein                           |
|

In [ ]:
["Energy",
"Carbohydrate, by difference",
"Water",
"Total lipid (fat)", # fat??
"Sugars, Total", # sugar: lactose, frutose, ...
"Protein",
"Fiber, total dietary"]

['Energy',
 'Carbohydrate, by difference',
 'Water',
 'Total lipid (fat)',
 'Sugars, Total',
 'Protein',
 'Fiber, total dietary']

In [ ]:
# Select the required fields from the DataFrame with alias to avoid column name duplication
df_selected = df.select(
    "description",
    "foodNutrients.amount",
    "foodNutrients.nutrient.name",
    "foodNutrients.nutrient.unitName",
    F.col("foodPortions.amount").alias("foodPortions_amount"),  # Đổi tên cột để tránh trùng lặp
    "foodPortions.gramWeight",
    "foodPortions.measureUnit.name"
)

# Convert to Pandas DataFrame
pandas_df = df_selected.toPandas()

# Show the result
pandas_df.head()


,description,amount,name,unitName,foodPortions_amount,gramWeight,name
0,None,None,None,None,None,None,None
1,"Hummus, commercial","[3.0, 0.0, 1.3, 0.0, 0.0, 0.0, 0.15, 0.115, 36...","[Cryptoxanthin, beta, Lycopene, Tocopherol, de...","[µg, µg, mg, mg, mg, mg, mg, mg, µg, µg, µg, g...",[2.0],[33.9],[tablespoon]
2,"Tomatoes, grape, raw","[92.5, 27.2, 0.56, 0.63, 0.13, 0.0, 8.0, 0.0, ...","[Water, Vitamin C, total ascorbic acid, Ash, T...","[g, mg, g, g, g, mg, mg, mg, mg, mg, mg, mg, m...","[5.0, 1.0]","[49.7, 152.0]","[tomatoes, cup]"
3,"Beans, snap, green, canned, regular pack, drai...","[0.0, 0.0, 93.6, 0.19, 36.0, 0.39, 4.11, 21.0,...","[Galactose, Lactose, Water, Zinc, Zn, Calcium,...","[g, g, g, mg, mg, g, g, kcal, g, g, kJ, mg, mg...",[1.0],[129.0],[cup]
4,"Frankfurter, beef, unheated","[28.0, 2.89, 314.0, 15.0, 343.0, 2.06, 1.17, 0...","[Total lipid (fat), Carbohydrate, by differenc...","[g, g, kcal, mg, mg, mg, g, g, g, mg, mg, mg, ...",[1.0],[48.6],[piece]


In [ ]:
pandas_df.to_csv("foundationIngredients.csv")

### Standardize data type

In [2]:
df = pd.read_csv("foundationIngredients.csv")
df = df.dropna()
for col in ["amount", "name", "unitName"]:
    df[col] = list(map(np.array, list(df[col])))
    df[col] = df[col].astype(str)
    if col in ["name", "unitName"]:
        f = lambda x: str(x).strip("'[]").split("', '")
        df[col] = list(map(f, df[col]))
        print(col)
    else:
    # except:
        # try:
            f = lambda x: str(x).strip("[]").split(", ")
            df[col] = list(map(f, df[col]))
            def f2(x):
                x_ = x.copy()
                for i in range(len(x)):
                    try:
                        x_[i] = float(x[i])
                    except:
                        x_[i] = 0.0
                        # print(i)
                        # print(x_)
                        # continue
                return x_
            # f2 = lambda x: [float(e) for e in x]
            df[col] = list(map(f2, df[col]))
            
        # print(col)
        
    # np.fromstring(array_string.strip('[]'), sep=',')
    # ast.literal_eval(string)
    df[col] = list(map(np.array, list(df[col])))
df = df.reset_index().drop(columns = ["index", "Unnamed: 0"])
df

name
unitName


,description,amount,name,unitName,foodPortions_amount,gramWeight,name.1
0,"Hummus, commercial","[3.0, 0.0, 1.3, 0.0, 0.0, 0.0, 0.15, 0.115, 36...","[Cryptoxanthin, beta, Lycopene, Tocopherol, de...","[µg, µg, mg, mg, mg, mg, mg, mg, µg, µg, µg, g...",[2.0],[33.9],['tablespoon']
1,"Tomatoes, grape, raw","[92.5, 27.2, 0.56, 0.63, 0.13, 0.0, 8.0, 0.0, ...","[Water, Vitamin C, total ascorbic acid, Ash, T...","[g, mg, g, g, g, mg, mg, mg, mg, mg, mg, mg, m...","[5.0, 1.0]","[49.7, 152.0]","['tomatoes', 'cup']"
2,"Beans, snap, green, canned, regular pack, drai...","[0.0, 0.0, 93.6, 0.19, 36.0, 0.39, 4.11, 21.0,...","[Galactose, Lactose, Water, Zinc, Zn, Calcium,...","[g, g, g, mg, mg, g, g, kcal, g, g, kJ, mg, mg...",[1.0],[129.0],['cup']
3,"Frankfurter, beef, unheated","[28.0, 2.89, 314.0, 15.0, 343.0, 2.06, 1.17, 0...","[Total lipid (fat), Carbohydrate, by differenc...","[g, g, kcal, mg, mg, mg, g, g, g, mg, mg, mg, ...",[1.0],[48.6],['piece']
4,"Nuts, almonds, dry roasted, with salt added","[0.005, 0.008, 0.0, 0.0, 0.0, 17.0, 0.0, 0.0, ...","[PUFA 20:4, SFA 15:0, PUFA 20:2 n-6 c,c, SFA 1...","[g, g, g, g, g, µg, µg, mg, g, g, g, g, g, g, ...",[1.0],[135.0],['cup']
...,...,...,...,...,...,...,...
311,"Sorghum bran, white, unenriched, dry, raw","[14.2, 274.0, 517.0, 852.0, 2.92, 4.4, 1.79, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[]
312,"Sorghum flour, white, pearled, unenriched, dry...","[2.22, 107.0, 246.0, 274.0, 0.0, 1.32, 1.63, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[]
313,"Sorghum grain, white, pearled, unenriched, dry...","[2.04, 109.0, 243.0, 274.0, 0.0, 1.28, 1.64, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[]
314,"Sorghum, whole grain, white, dry, raw","[3.9, 136.0, 294.0, 367.0, 0.329, 1.77, 1.61, ...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[]


In [3]:
df[df["description"] == "Plantains, overripe, raw"]["name"].values

array([array(['Iron, Fe', 'Magnesium, Mg', 'Phosphorus, P', 'Potassium, K',
              'Sodium, Na', 'Zinc, Zn', 'Nitrogen', 'Copper, Cu',
              'Vitamin C, total ascorbic acid', 'Total lipid (fat)',
              'Manganese, Mn', 'Riboflavin', 'Ash', 'Starch', 'Sucrose',
              'Glucose', 'Fructose', 'Lactose', 'Maltose',
              'Fiber, total dietary', 'Vitamin B-6', 'Water', 'Calcium, Ca',
              'Energy (Atwater General Factors)', 'Protein',
              'Carbohydrate, by difference', 'Energy (Atwater Specific Factors)',
              'Sugars, Total'], dtype='<U33')                                    ],
      dtype=object)

### Filter nutrients

In [9]:
filtered_nutrients = ["Energy",
"Carbohydrate, by difference",
"Water",
"Total lipid (fat)", # fat??
"Sugars, Total", # sugar: lactose, frutose, ...
"Protein",
"Fiber, total dietary"]

In [10]:
# mask = [True if nutrient in nutrients else False for nutrient in df.iloc[x]['name']]
# return_mask = lambda list_nutrient: [True if nutrient in filtered_nutrients else False for nutrient in list_nutrient]
return_mask = lambda list_nutrient, unit_list: [
    True if nutrient in filtered_nutrients and unit != 'kJ' else False 
    for nutrient, unit in zip(list_nutrient, unit_list)
]

df["mask"] = list(map(return_mask, df["name"], df["unitName"]))
# df["mask"] = list(map(return_mask, df["name"]))
df["mask"] = list(map(np.array, list(df["mask"])))
# x ~ row xth
df

,description,amount,name,unitName,foodPortions_amount,gramWeight,name.1,mask
0,"Hummus, commercial","[3.0, 0.0, 1.3, 0.0, 0.0, 0.0, 0.15, 0.115, 36...","[Cryptoxanthin, beta, Lycopene, Tocopherol, de...","[µg, µg, mg, mg, mg, mg, mg, mg, µg, µg, µg, g...",[2.0],[33.9],['tablespoon'],"[False, False, False, False, False, False, Fal..."
1,"Tomatoes, grape, raw","[92.5, 27.2, 0.56, 0.63, 0.13, 0.0, 8.0, 0.0, ...","[Water, Vitamin C, total ascorbic acid, Ash, T...","[g, mg, g, g, g, mg, mg, mg, mg, mg, mg, mg, m...","[5.0, 1.0]","[49.7, 152.0]","['tomatoes', 'cup']","[True, False, False, True, False, False, False..."
2,"Beans, snap, green, canned, regular pack, drai...","[0.0, 0.0, 93.6, 0.19, 36.0, 0.39, 4.11, 21.0,...","[Galactose, Lactose, Water, Zinc, Zn, Calcium,...","[g, g, g, mg, mg, g, g, kcal, g, g, kJ, mg, mg...",[1.0],[129.0],['cup'],"[False, False, True, False, False, True, True,..."
3,"Frankfurter, beef, unheated","[28.0, 2.89, 314.0, 15.0, 343.0, 2.06, 1.17, 0...","[Total lipid (fat), Carbohydrate, by differenc...","[g, g, kcal, mg, mg, mg, g, g, g, mg, mg, mg, ...",[1.0],[48.6],['piece'],"[True, True, True, False, False, False, False,..."
4,"Nuts, almonds, dry roasted, with salt added","[0.005, 0.008, 0.0, 0.0, 0.0, 17.0, 0.0, 0.0, ...","[PUFA 20:4, SFA 15:0, PUFA 20:2 n-6 c,c, SFA 1...","[g, g, g, g, g, µg, µg, mg, g, g, g, g, g, g, ...",[1.0],[135.0],['cup'],"[False, False, False, False, False, False, Fal..."
...,...,...,...,...,...,...,...,...
311,"Sorghum bran, white, unenriched, dry, raw","[14.2, 274.0, 517.0, 852.0, 2.92, 4.4, 1.79, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal..."
312,"Sorghum flour, white, pearled, unenriched, dry...","[2.22, 107.0, 246.0, 274.0, 0.0, 1.32, 1.63, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal..."
313,"Sorghum grain, white, pearled, unenriched, dry...","[2.04, 109.0, 243.0, 274.0, 0.0, 1.28, 1.64, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal..."
314,"Sorghum, whole grain, white, dry, raw","[3.9, 136.0, 294.0, 367.0, 0.329, 1.77, 1.61, ...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal..."


In [11]:
import numpy as np

# Define filtered nutrients
filtered_nutrients = [
    "Energy", # "Energy", "Energy (Atwater General Factors)", "Energy (Atwater General Factors)
    "Carbohydrate, by difference",
    "Water",
    "Total lipid (fat)",  # fat
    "Sugars, Total",  # sugar: lactose, fructose, etc.
    "Protein",
    "Fiber, total dietary"
]

# Function to filter and sum Energy (KCAL)
def return_mask(list_nutrient, unit_list, amount_list):
    # Sum all 'Energy' with 'kcal' units
    energy_kcal_sum = sum(
        amount for nutrient, unit, amount in zip(list_nutrient, unit_list, amount_list)
        if 'Energy' in nutrient and unit == 'kcal'
    )
    
    # Create mask
    mask = [
        True if nutrient in filtered_nutrients and unit != 'kJ' else False 
        for nutrient, unit in zip(list_nutrient, unit_list)
    ]
    
    # If 'Energy' is in the list and has 'kcal', set its value to the sum of all Energy (KCAL)
    if any('Energy' in nutrient for nutrient in list_nutrient):
        print([nutrient for nutrient in list_nutrient if 'Energy' in nutrient])
        mask = ['Energy' if 'Energy' in nutrient and unit != 'kJ' else m for nutrient, unit, m in zip(list_nutrient, unit_list, mask)]
                # for nutrient, m in zip(list_nutrient, mask)]
        
    #         True if nutrient in filtered_nutrients and unit != 'kJ' else False 
    # for nutrient, unit in zip(list_nutrient, unit_list)
    
    return mask

# Apply return_mask function to create a new column with the mask
df["mask"] = list(map(return_mask, df["name"], df["unitName"], df["amount"]))
df["mask"] = list(map(np.array, list(df["mask"])))

# Example output
df

['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'Energy']
['Energy', 'E

,description,amount,name,unitName,foodPortions_amount,gramWeight,name.1,mask
0,"Hummus, commercial","[3.0, 0.0, 1.3, 0.0, 0.0, 0.0, 0.15, 0.115, 36...","[Cryptoxanthin, beta, Lycopene, Tocopherol, de...","[µg, µg, mg, mg, mg, mg, mg, mg, µg, µg, µg, g...",[2.0],[33.9],['tablespoon'],"[False, False, False, False, False, False, Fal..."
1,"Tomatoes, grape, raw","[92.5, 27.2, 0.56, 0.63, 0.13, 0.0, 8.0, 0.0, ...","[Water, Vitamin C, total ascorbic acid, Ash, T...","[g, mg, g, g, g, mg, mg, mg, mg, mg, mg, mg, m...","[5.0, 1.0]","[49.7, 152.0]","['tomatoes', 'cup']","[True, False, False, True, False, False, False..."
2,"Beans, snap, green, canned, regular pack, drai...","[0.0, 0.0, 93.6, 0.19, 36.0, 0.39, 4.11, 21.0,...","[Galactose, Lactose, Water, Zinc, Zn, Calcium,...","[g, g, g, mg, mg, g, g, kcal, g, g, kJ, mg, mg...",[1.0],[129.0],['cup'],"[False, False, True, False, False, True, True,..."
3,"Frankfurter, beef, unheated","[28.0, 2.89, 314.0, 15.0, 343.0, 2.06, 1.17, 0...","[Total lipid (fat), Carbohydrate, by differenc...","[g, g, kcal, mg, mg, mg, g, g, g, mg, mg, mg, ...",[1.0],[48.6],['piece'],"[True, True, Energy, False, False, False, Fals..."
4,"Nuts, almonds, dry roasted, with salt added","[0.005, 0.008, 0.0, 0.0, 0.0, 17.0, 0.0, 0.0, ...","[PUFA 20:4, SFA 15:0, PUFA 20:2 n-6 c,c, SFA 1...","[g, g, g, g, g, µg, µg, mg, g, g, g, g, g, g, ...",[1.0],[135.0],['cup'],"[False, False, False, False, False, False, Fal..."
...,...,...,...,...,...,...,...,...
311,"Sorghum bran, white, unenriched, dry, raw","[14.2, 274.0, 517.0, 852.0, 2.92, 4.4, 1.79, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal..."
312,"Sorghum flour, white, pearled, unenriched, dry...","[2.22, 107.0, 246.0, 274.0, 0.0, 1.32, 1.63, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal..."
313,"Sorghum grain, white, pearled, unenriched, dry...","[2.04, 109.0, 243.0, 274.0, 0.0, 1.28, 1.64, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal..."
314,"Sorghum, whole grain, white, dry, raw","[3.9, 136.0, 294.0, 367.0, 0.329, 1.77, 1.61, ...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal..."


In [12]:
# Calculate averages for elements in 'amount' where mask indicates 'Energy'
energy_averages = []

# Loop through each row and calculate the average
for index, row in df.iterrows():
    amounts = row['amount']
    masks = row['mask']
    
    # Get only the amounts where mask indicates Energy
    energy_amounts = [amounts[i] for i in range(len(masks)) if masks[i] == 'Energy']
    print(energy_amounts)
    
    # Calculate average if there are any energy amounts
    if energy_amounts:
        average = sum(energy_amounts) / len(energy_amounts)
    else:
        average = None  # or use 0 if you want to replace None with a default value
    
    energy_averages.append(average)

# Add the average values to the DataFrame
df['energy_average'] = energy_averages

# Display the updated DataFrame
df

[229.0]
[27.0]
[21.0]
[314.0]
[620.0]
[35.0]
[150.0]
[48.0]
[376.0]
[288.0]
[12.0]
[421.0]
[366.0]
[37.0]
[42.0]
[612.0]
[270.0]
[36.0]
[61.0]
[58.0]
[39.0]
[408.0]
[84.0]
[298.0]
[575.0]
[296.0]
[654.0]
[61.0]
[83.0]
[833.0]
[156.0]
[166.0]
[45.0]
[106.0]
[130.0]
[430.0]
[18.0]
[74.0]
[56.0]
[90.0]
[174.0]
[174.0]
[229.0]
[254.0]
[176.0]
[155.0]
[122.0]
[123.0]
[145.0]
[219.0]
[37.0]
[326.0]
[157.0]
[393.0]
[249.0]
[17.0]
[34.0]
[47.0]
[43.0]
[57.0]
[260.0]
[]
[34.0]
[29.0]
[50.0]
[328.0]
[322.0]
[346.0]
[60.0]
[169.0]
[385.0]
[220.0]
[121.0]
[375.0]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[31.0]
[117.0]
[55.0]
[334.0]
[]
[]
[]
[]
[148.0]
[500.0]
[]
[366.0]
[358.0]
[362.0]
[370.0]
[363.0]
[359.0]
[364.0]
[]
[44.0]
[38.0]
[143.0]
[366.0]
[452.0]
[365.0]
[358.0]
[358.0]
[35.0]
[85.0]
[97.0]
[61.8, 55.6]
[64.7, 58.2]
[61.0, 54.9]
[58.9, 53.0]
[60.0, 54.0]
[]
[]
[]
[]
[42.7, 35.1]
[41.0, 33.3]
[44.1, 36.4]
[31.2, 24.9]
[38.5, 38.5]
[14.6, 14.6]
[26.6, 20.7]
[27.6, 21.6]
[22.0, 

,description,amount,name,unitName,foodPortions_amount,gramWeight,name.1,mask,energy_average
0,"Hummus, commercial","[3.0, 0.0, 1.3, 0.0, 0.0, 0.0, 0.15, 0.115, 36...","[Cryptoxanthin, beta, Lycopene, Tocopherol, de...","[µg, µg, mg, mg, mg, mg, mg, mg, µg, µg, µg, g...",[2.0],[33.9],['tablespoon'],"[False, False, False, False, False, False, Fal...",229.0
1,"Tomatoes, grape, raw","[92.5, 27.2, 0.56, 0.63, 0.13, 0.0, 8.0, 0.0, ...","[Water, Vitamin C, total ascorbic acid, Ash, T...","[g, mg, g, g, g, mg, mg, mg, mg, mg, mg, mg, m...","[5.0, 1.0]","[49.7, 152.0]","['tomatoes', 'cup']","[True, False, False, True, False, False, False...",27.0
2,"Beans, snap, green, canned, regular pack, drai...","[0.0, 0.0, 93.6, 0.19, 36.0, 0.39, 4.11, 21.0,...","[Galactose, Lactose, Water, Zinc, Zn, Calcium,...","[g, g, g, mg, mg, g, g, kcal, g, g, kJ, mg, mg...",[1.0],[129.0],['cup'],"[False, False, True, False, False, True, True,...",21.0
3,"Frankfurter, beef, unheated","[28.0, 2.89, 314.0, 15.0, 343.0, 2.06, 1.17, 0...","[Total lipid (fat), Carbohydrate, by differenc...","[g, g, kcal, mg, mg, mg, g, g, g, mg, mg, mg, ...",[1.0],[48.6],['piece'],"[True, True, Energy, False, False, False, Fals...",314.0
4,"Nuts, almonds, dry roasted, with salt added","[0.005, 0.008, 0.0, 0.0, 0.0, 17.0, 0.0, 0.0, ...","[PUFA 20:4, SFA 15:0, PUFA 20:2 n-6 c,c, SFA 1...","[g, g, g, g, g, µg, µg, mg, g, g, g, g, g, g, ...",[1.0],[135.0],['cup'],"[False, False, False, False, False, False, Fal...",620.0
...,...,...,...,...,...,...,...,...,...
311,"Sorghum bran, white, unenriched, dry, raw","[14.2, 274.0, 517.0, 852.0, 2.92, 4.4, 1.79, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",384.0
312,"Sorghum flour, white, pearled, unenriched, dry...","[2.22, 107.0, 246.0, 274.0, 0.0, 1.32, 1.63, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",357.0
313,"Sorghum grain, white, pearled, unenriched, dry...","[2.04, 109.0, 243.0, 274.0, 0.0, 1.28, 1.64, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",354.0
314,"Sorghum, whole grain, white, dry, raw","[3.9, 136.0, 294.0, 367.0, 0.329, 1.77, 1.61, ...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",357.0


In [13]:
df = df.dropna()
df.reset_index().drop(columns = ["index"])

,description,amount,name,unitName,foodPortions_amount,gramWeight,name.1,mask,energy_average
0,"Hummus, commercial","[3.0, 0.0, 1.3, 0.0, 0.0, 0.0, 0.15, 0.115, 36...","[Cryptoxanthin, beta, Lycopene, Tocopherol, de...","[µg, µg, mg, mg, mg, mg, mg, mg, µg, µg, µg, g...",[2.0],[33.9],['tablespoon'],"[False, False, False, False, False, False, Fal...",229.0
1,"Tomatoes, grape, raw","[92.5, 27.2, 0.56, 0.63, 0.13, 0.0, 8.0, 0.0, ...","[Water, Vitamin C, total ascorbic acid, Ash, T...","[g, mg, g, g, g, mg, mg, mg, mg, mg, mg, mg, m...","[5.0, 1.0]","[49.7, 152.0]","['tomatoes', 'cup']","[True, False, False, True, False, False, False...",27.0
2,"Beans, snap, green, canned, regular pack, drai...","[0.0, 0.0, 93.6, 0.19, 36.0, 0.39, 4.11, 21.0,...","[Galactose, Lactose, Water, Zinc, Zn, Calcium,...","[g, g, g, mg, mg, g, g, kcal, g, g, kJ, mg, mg...",[1.0],[129.0],['cup'],"[False, False, True, False, False, True, True,...",21.0
3,"Frankfurter, beef, unheated","[28.0, 2.89, 314.0, 15.0, 343.0, 2.06, 1.17, 0...","[Total lipid (fat), Carbohydrate, by differenc...","[g, g, kcal, mg, mg, mg, g, g, g, mg, mg, mg, ...",[1.0],[48.6],['piece'],"[True, True, Energy, False, False, False, Fals...",314.0
4,"Nuts, almonds, dry roasted, with salt added","[0.005, 0.008, 0.0, 0.0, 0.0, 17.0, 0.0, 0.0, ...","[PUFA 20:4, SFA 15:0, PUFA 20:2 n-6 c,c, SFA 1...","[g, g, g, g, g, µg, µg, mg, g, g, g, g, g, g, ...",[1.0],[135.0],['cup'],"[False, False, False, False, False, False, Fal...",620.0
...,...,...,...,...,...,...,...,...,...
281,"Sorghum bran, white, unenriched, dry, raw","[14.2, 274.0, 517.0, 852.0, 2.92, 4.4, 1.79, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",384.0
282,"Sorghum flour, white, pearled, unenriched, dry...","[2.22, 107.0, 246.0, 274.0, 0.0, 1.32, 1.63, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",357.0
283,"Sorghum grain, white, pearled, unenriched, dry...","[2.04, 109.0, 243.0, 274.0, 0.0, 1.28, 1.64, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",354.0
284,"Sorghum, whole grain, white, dry, raw","[3.9, 136.0, 294.0, 367.0, 0.329, 1.77, 1.61, ...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",357.0


In [14]:
# df['mask'] = df['mask'].astype(bool)
# df["amount"][df['mask']]

# f3 = lambda x, y: [x[i] if (y[i] == True) for i in range(len(x))]
f3 = lambda x, y: [x[i] for i in range(len(x)) if y[i] == 'True']

cols = ["amount", "name", "unitName"]
for col in cols:
    df[f"{col}_"] = list(map(f3, df[col], df['mask']))
    



C:\Users\Administrator\AppData\Local\Temp\ipykernel_20064\1741902629.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_"] = list(map(f3, df[col], df['mask']))


In [15]:
df.iloc[1]["mask"]

array(['True', 'False', 'False', 'True', 'False', 'False', 'False',
       'False', 'False', 'False', 'False', 'False', 'False', 'False',
       'False', 'False', 'False', 'False', 'False', 'False', 'False',
       'True', 'False', 'False', 'False', 'False', 'False', 'False',
       'False', 'False', 'False', 'False', 'False', 'False', 'False',
       'False', 'False', 'False', 'False', 'False', 'False', 'False',
       'False', 'False', 'False', 'False', 'False', 'False', 'True',
       'True', 'Energy', 'False', 'False'], dtype='<U6')

In [16]:
df

,description,amount,name,unitName,foodPortions_amount,gramWeight,name.1,mask,energy_average,amount_,name_,unitName_
0,"Hummus, commercial","[3.0, 0.0, 1.3, 0.0, 0.0, 0.0, 0.15, 0.115, 36...","[Cryptoxanthin, beta, Lycopene, Tocopherol, de...","[µg, µg, mg, mg, mg, mg, mg, mg, µg, µg, µg, g...",[2.0],[33.9],['tablespoon'],"[False, False, False, False, False, False, Fal...",229.0,"[7.35, 5.4, 17.1, 14.9, 58.7, 0.34]","[Protein, Fiber, total dietary, Total lipid (f...","[g, g, g, g, g, g]"
1,"Tomatoes, grape, raw","[92.5, 27.2, 0.56, 0.63, 0.13, 0.0, 8.0, 0.0, ...","[Water, Vitamin C, total ascorbic acid, Ash, T...","[g, mg, g, g, g, mg, mg, mg, mg, mg, mg, mg, m...","[5.0, 1.0]","[49.7, 152.0]","['tomatoes', 'cup']","[True, False, False, True, False, False, False...",27.0,"[92.5, 0.63, 2.1, 0.83, 5.51]","[Water, Total lipid (fat), Fiber, total dietar...","[g, g, g, g, g]"
2,"Beans, snap, green, canned, regular pack, drai...","[0.0, 0.0, 93.6, 0.19, 36.0, 0.39, 4.11, 21.0,...","[Galactose, Lactose, Water, Zinc, Zn, Calcium,...","[g, g, g, mg, mg, g, g, kcal, g, g, kJ, mg, mg...",[1.0],[129.0],['cup'],"[False, False, True, False, False, True, True,...",21.0,"[93.6, 0.39, 4.11, 1.04, 1.29]","[Water, Total lipid (fat), Carbohydrate, by di...","[g, g, g, g, g]"
3,"Frankfurter, beef, unheated","[28.0, 2.89, 314.0, 15.0, 343.0, 2.06, 1.17, 0...","[Total lipid (fat), Carbohydrate, by differenc...","[g, g, kcal, mg, mg, mg, g, g, g, mg, mg, mg, ...",[1.0],[48.6],['piece'],"[True, True, Energy, False, False, False, Fals...",314.0,"[28.0, 2.89, 54.6, 11.7, 1.26]","[Total lipid (fat), Carbohydrate, by differenc...","[g, g, g, g, g]"
4,"Nuts, almonds, dry roasted, with salt added","[0.005, 0.008, 0.0, 0.0, 0.0, 17.0, 0.0, 0.0, ...","[PUFA 20:4, SFA 15:0, PUFA 20:2 n-6 c,c, SFA 1...","[g, g, g, g, g, µg, µg, mg, g, g, g, g, g, g, ...",[1.0],[135.0],['cup'],"[False, False, False, False, False, False, Fal...",620.0,"[57.8, 16.2, 2.2, 20.4, 11.0, 4.17]","[Total lipid (fat), Carbohydrate, by differenc...","[g, g, g, g, g, g]"
...,...,...,...,...,...,...,...,...,...,...,...,...
311,"Sorghum bran, white, unenriched, dry, raw","[14.2, 274.0, 517.0, 852.0, 2.92, 4.4, 1.79, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",384.0,"[9.26, 35.0, 7.42, 11.2, 68.7]","[Total lipid (fat), Fiber, total dietary, Wate...","[g, g, g, g, g]"
312,"Sorghum flour, white, pearled, unenriched, dry...","[2.22, 107.0, 246.0, 274.0, 0.0, 1.32, 1.63, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",357.0,"[3.24, 3.3, 11.9, 10.2, 73.5]","[Total lipid (fat), Fiber, total dietary, Wate...","[g, g, g, g, g]"
313,"Sorghum grain, white, pearled, unenriched, dry...","[2.04, 109.0, 243.0, 274.0, 0.0, 1.28, 1.64, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",354.0,"[3.26, 3.92, 10.4, 10.2, 74.9]","[Total lipid (fat), Fiber, total dietary, Wate...","[g, g, g, g, g]"
314,"Sorghum, whole grain, white, dry, raw","[3.9, 136.0, 294.0, 367.0, 0.329, 1.77, 1.61, ...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",357.0,"[4.22, 8.27, 10.6, 73.6, 10.1]","[Total lipid (fat), Fiber, total dietary, Wate...","[g, g, g, g, g]"


In [17]:
df[["description","amount_", "name_",	"unitName_", "energy_average"]]

,description,amount_,name_,unitName_,energy_average
0,"Hummus, commercial","[7.35, 5.4, 17.1, 14.9, 58.7, 0.34]","[Protein, Fiber, total dietary, Total lipid (f...","[g, g, g, g, g, g]",229.0
1,"Tomatoes, grape, raw","[92.5, 0.63, 2.1, 0.83, 5.51]","[Water, Total lipid (fat), Fiber, total dietar...","[g, g, g, g, g]",27.0
2,"Beans, snap, green, canned, regular pack, drai...","[93.6, 0.39, 4.11, 1.04, 1.29]","[Water, Total lipid (fat), Carbohydrate, by di...","[g, g, g, g, g]",21.0
3,"Frankfurter, beef, unheated","[28.0, 2.89, 54.6, 11.7, 1.26]","[Total lipid (fat), Carbohydrate, by differenc...","[g, g, g, g, g]",314.0
4,"Nuts, almonds, dry roasted, with salt added","[57.8, 16.2, 2.2, 20.4, 11.0, 4.17]","[Total lipid (fat), Carbohydrate, by differenc...","[g, g, g, g, g, g]",620.0
...,...,...,...,...,...
311,"Sorghum bran, white, unenriched, dry, raw","[9.26, 35.0, 7.42, 11.2, 68.7]","[Total lipid (fat), Fiber, total dietary, Wate...","[g, g, g, g, g]",384.0
312,"Sorghum flour, white, pearled, unenriched, dry...","[3.24, 3.3, 11.9, 10.2, 73.5]","[Total lipid (fat), Fiber, total dietary, Wate...","[g, g, g, g, g]",357.0
313,"Sorghum grain, white, pearled, unenriched, dry...","[3.26, 3.92, 10.4, 10.2, 74.9]","[Total lipid (fat), Fiber, total dietary, Wate...","[g, g, g, g, g]",354.0
314,"Sorghum, whole grain, white, dry, raw","[4.22, 8.27, 10.6, 73.6, 10.1]","[Total lipid (fat), Fiber, total dietary, Wate...","[g, g, g, g, g]",357.0


In [18]:
# Step 1: Add 'average_energy' to the 'name_' column
df['name_'] = df['name_'].apply(lambda x: x + ['average_energy'])

# Step 2: Add the corresponding 'energy_average' value to the 'amount_' column
df['amount_'] = df.apply(lambda row: row['amount_'] + [row['energy_average']], axis=1)

df['unitName_'] = df['unitName_'].apply(lambda x: x + ['kcal'])

# Display the updated DataFrame
df

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20064\3608891419.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['name_'] = df['name_'].apply(lambda x: x + ['average_energy'])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20064\3608891419.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['amount_'] = df.apply(lambda row: row['amount_'] + [row['energy_average']], axis=1)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20064\3608891419.py:8: SettingWithCopyWarning: 
A value is trying

,description,amount,name,unitName,foodPortions_amount,gramWeight,name.1,mask,energy_average,amount_,name_,unitName_
0,"Hummus, commercial","[3.0, 0.0, 1.3, 0.0, 0.0, 0.0, 0.15, 0.115, 36...","[Cryptoxanthin, beta, Lycopene, Tocopherol, de...","[µg, µg, mg, mg, mg, mg, mg, mg, µg, µg, µg, g...",[2.0],[33.9],['tablespoon'],"[False, False, False, False, False, False, Fal...",229.0,"[7.35, 5.4, 17.1, 14.9, 58.7, 0.34, 229.0]","[Protein, Fiber, total dietary, Total lipid (f...","[g, g, g, g, g, g, kcal]"
1,"Tomatoes, grape, raw","[92.5, 27.2, 0.56, 0.63, 0.13, 0.0, 8.0, 0.0, ...","[Water, Vitamin C, total ascorbic acid, Ash, T...","[g, mg, g, g, g, mg, mg, mg, mg, mg, mg, mg, m...","[5.0, 1.0]","[49.7, 152.0]","['tomatoes', 'cup']","[True, False, False, True, False, False, False...",27.0,"[92.5, 0.63, 2.1, 0.83, 5.51, 27.0]","[Water, Total lipid (fat), Fiber, total dietar...","[g, g, g, g, g, kcal]"
2,"Beans, snap, green, canned, regular pack, drai...","[0.0, 0.0, 93.6, 0.19, 36.0, 0.39, 4.11, 21.0,...","[Galactose, Lactose, Water, Zinc, Zn, Calcium,...","[g, g, g, mg, mg, g, g, kcal, g, g, kJ, mg, mg...",[1.0],[129.0],['cup'],"[False, False, True, False, False, True, True,...",21.0,"[93.6, 0.39, 4.11, 1.04, 1.29, 21.0]","[Water, Total lipid (fat), Carbohydrate, by di...","[g, g, g, g, g, kcal]"
3,"Frankfurter, beef, unheated","[28.0, 2.89, 314.0, 15.0, 343.0, 2.06, 1.17, 0...","[Total lipid (fat), Carbohydrate, by differenc...","[g, g, kcal, mg, mg, mg, g, g, g, mg, mg, mg, ...",[1.0],[48.6],['piece'],"[True, True, Energy, False, False, False, Fals...",314.0,"[28.0, 2.89, 54.6, 11.7, 1.26, 314.0]","[Total lipid (fat), Carbohydrate, by differenc...","[g, g, g, g, g, kcal]"
4,"Nuts, almonds, dry roasted, with salt added","[0.005, 0.008, 0.0, 0.0, 0.0, 17.0, 0.0, 0.0, ...","[PUFA 20:4, SFA 15:0, PUFA 20:2 n-6 c,c, SFA 1...","[g, g, g, g, g, µg, µg, mg, g, g, g, g, g, g, ...",[1.0],[135.0],['cup'],"[False, False, False, False, False, False, Fal...",620.0,"[57.8, 16.2, 2.2, 20.4, 11.0, 4.17, 620.0]","[Total lipid (fat), Carbohydrate, by differenc...","[g, g, g, g, g, g, kcal]"
...,...,...,...,...,...,...,...,...,...,...,...,...
311,"Sorghum bran, white, unenriched, dry, raw","[14.2, 274.0, 517.0, 852.0, 2.92, 4.4, 1.79, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",384.0,"[9.26, 35.0, 7.42, 11.2, 68.7, 384.0]","[Total lipid (fat), Fiber, total dietary, Wate...","[g, g, g, g, g, kcal]"
312,"Sorghum flour, white, pearled, unenriched, dry...","[2.22, 107.0, 246.0, 274.0, 0.0, 1.32, 1.63, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",357.0,"[3.24, 3.3, 11.9, 10.2, 73.5, 357.0]","[Total lipid (fat), Fiber, total dietary, Wate...","[g, g, g, g, g, kcal]"
313,"Sorghum grain, white, pearled, unenriched, dry...","[2.04, 109.0, 243.0, 274.0, 0.0, 1.28, 1.64, 0...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",354.0,"[3.26, 3.92, 10.4, 10.2, 74.9, 354.0]","[Total lipid (fat), Fiber, total dietary, Wate...","[g, g, g, g, g, kcal]"
314,"Sorghum, whole grain, white, dry, raw","[3.9, 136.0, 294.0, 367.0, 0.329, 1.77, 1.61, ...","[Iron, Fe, Magnesium, Mg, Phosphorus, P, Potas...","[mg, mg, mg, mg, mg, mg, g, mg, g, mg, mg, mg,...",[],[],[],"[False, False, False, False, False, False, Fal...",357.0,"[4.22, 8.27, 10.6, 73.6, 10.1, 357.0]","[Total lipid (fat), Fiber, total dietary, Wate...","[g, g, g, g, g, kcal]"


### Map ingredient name to output of ingredient detection model

In [19]:
names = ['almond', 'apple', 'asparagus', 'avocado', 'bacon', 'banana', 'bean', 'bean sprout', 'beef', 'beetroot', 'bell pepper', 'blackberry', 'blueberry', 'bok choy', 'bread', 'brie cheese', 'broccoli', 'cabbage', 'carrot', 'cauliflower', 'cheddar cheese', 'cheese', 'cherry', 'chicken breast', 'chicken wing', 'chilli', 'chocolate', 'corn', 'cucumber', 'dry grape', 'durian', 'egg', 'eggplant', 'fish', 'garlic', 'ginger', 'grape', 'green grape', 'green pepper', 'guava', 'jalepeno', 'jam', 'kiwi', 'lemon', 'mango', 'mangoteen', 'meat ball', 'milk', 'mozarella cheese', 'mushroom', 'mussel', 'noodle', 'onion', 'orange', 'oyster', 'papaya', 'parmesan cheese', 'pasta', 'pineapple', 'pomegranate', 'pork', 'pork belly', 'pork rib', 'potato', 'pumpkin', 'raspberry', 'salad', 'salmon', 'scallop', 'shrimp', 'spring onion', 'starfruit', 'stilton cheese', 'strawberry', 'sweet potato', 'tomato', 'tuna', 'vegetable', 'watermelon', 'yogurt']
names

['almond',
 'apple',
 'asparagus',
 'avocado',
 'bacon',
 'banana',
 'bean',
 'bean sprout',
 'beef',
 'beetroot',
 'bell pepper',
 'blackberry',
 'blueberry',
 'bok choy',
 'bread',
 'brie cheese',
 'broccoli',
 'cabbage',
 'carrot',
 'cauliflower',
 'cheddar cheese',
 'cheese',
 'cherry',
 'chicken breast',
 'chicken wing',
 'chilli',
 'chocolate',
 'corn',
 'cucumber',
 'dry grape',
 'durian',
 'egg',
 'eggplant',
 'fish',
 'garlic',
 'ginger',
 'grape',
 'green grape',
 'green pepper',
 'guava',
 'jalepeno',
 'jam',
 'kiwi',
 'lemon',
 'mango',
 'mangoteen',
 'meat ball',
 'milk',
 'mozarella cheese',
 'mushroom',
 'mussel',
 'noodle',
 'onion',
 'orange',
 'oyster',
 'papaya',
 'parmesan cheese',
 'pasta',
 'pineapple',
 'pomegranate',
 'pork',
 'pork belly',
 'pork rib',
 'potato',
 'pumpkin',
 'raspberry',
 'salad',
 'salmon',
 'scallop',
 'shrimp',
 'spring onion',
 'starfruit',
 'stilton cheese',
 'strawberry',
 'sweet potato',
 'tomato',
 'tuna',
 'vegetable',
 'watermelon',


In [20]:
# Define the filtered nutrients to consider, setting any missing values to zero
filtered_nutrients = ["average_energy", "Carbohydrate, by difference", "Water", 
                      "Total lipid (fat)", "Sugars, Total", "Protein", "Fiber, total dietary"]

# Process each entry in the dataframe to ensure all filtered nutrients are present with missing ones set to 0
def complete_nutrient_amounts(amount_list, nutrient_list):
    # Create a dictionary from current nutrient names and amounts
    nutrient_dict = dict(zip(nutrient_list, amount_list))
    # Add missing nutrients with a value of 0
    for nutrient in filtered_nutrients:
        if nutrient not in nutrient_dict:
            nutrient_dict[nutrient] = 0.0
    # Return the amounts in the order of filtered_nutrients
    # return [nutrient_dict[nutrient] for nutrient in filtered_nutrients]
    return nutrient_dict

# Apply the completion function to each row
# df["completed_amount_"] = df.apply(lambda row: complete_nutrient_amounts(row["amount_"], row["name_"]), axis=1)
df["completed_amount_"] = df.apply(lambda row: complete_nutrient_amounts(row["amount_"], row["name_"]), axis=1)

# Display the updated dataframe to verify
df[["description", "completed_amount_"]].reset_index().drop(columns = ["index"])


C:\Users\Administrator\AppData\Local\Temp\ipykernel_20064\4104229806.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["completed_amount_"] = df.apply(lambda row: complete_nutrient_amounts(row["amount_"], row["name_"]), axis=1)


,description,completed_amount_
0,"Hummus, commercial","{'Protein': 7.35, 'Fiber, total dietary': 5.4,..."
1,"Tomatoes, grape, raw","{'Water': 92.5, 'Total lipid (fat)': 0.63, 'Fi..."
2,"Beans, snap, green, canned, regular pack, drai...","{'Water': 93.6, 'Total lipid (fat)': 0.39, 'Ca..."
3,"Frankfurter, beef, unheated","{'Total lipid (fat)': 28.0, 'Carbohydrate, by ..."
4,"Nuts, almonds, dry roasted, with salt added","{'Total lipid (fat)': 57.8, 'Carbohydrate, by ..."
...,...,...
281,"Sorghum bran, white, unenriched, dry, raw","{'Total lipid (fat)': 9.26, 'Fiber, total diet..."
282,"Sorghum flour, white, pearled, unenriched, dry...","{'Total lipid (fat)': 3.24, 'Fiber, total diet..."
283,"Sorghum grain, white, pearled, unenriched, dry...","{'Total lipid (fat)': 3.26, 'Fiber, total diet..."
284,"Sorghum, whole grain, white, dry, raw","{'Total lipid (fat)': 4.22, 'Fiber, total diet..."


In [21]:
'Flour, almond'
df[df["description"] == 'Flour, almond']["completed_amount_"].values

array([{'Total lipid (fat)': 50.2, 'Fiber, total dietary': 9.27, 'Water': 4.32, 'Protein': 26.2, 'Carbohydrate, by difference': 16.2, 'average_energy': 600.0, 'Sugars, Total': 0.0}],
      dtype=object)

In [22]:
df[["description", "completed_amount_", "name_"]].iloc[1]["completed_amount_"]

{'Water': 92.5,
 'Total lipid (fat)': 0.63,
 'Fiber, total dietary': 2.1,
 'Protein': 0.83,
 'Carbohydrate, by difference': 5.51,
 'average_energy': 27.0,
 'Sugars, Total': 0.0}

In [28]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
import nltk

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def tfidf_similarity(text1, text2):
    # Lemmatize the input texts
    text1_lemmatized = ' '.join(lemmatizer.lemmatize(word) for word in text1.lower().split())
    text2_lemmatized = ' '.join(lemmatizer.lemmatize(word) for word in text2.lower().split())
    
    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()
    
    # Fit and transform the texts
    X = vectorizer.fit_transform([text1_lemmatized, text2_lemmatized])
    
    # Calculate the cosine similarity
    from scipy.spatial.distance import cosine
    return 1 - cosine(X.toarray()[0], X.toarray()[1])


In [32]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [51]:
results = dict()

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = word_tokenize(text.lower())
    return ' '.join(lemmatizer.lemmatize(token) for token in tokens)

# Update your similarity calculation loop
for name in names:
    name_tokens = lemmatize_text(name)
    
    similarities = []
    for desc in df["description"]:
        desc_tokens = lemmatize_text(desc)
        similarity = tfidf_similarity(name_tokens, desc_tokens)
        similarities.append(similarity)
    
    max_similarity = max(similarities)
    best_match_indices = [i for i, sim in enumerate(similarities) if sim == max_similarity]

    # Xử lý averaged_nutrients và thêm foodPortions_amount, gramWeight, name.1
    if 15 > len(best_match_indices) > 1:
        avg_nutrient_dict = {
            nutrient: round(np.mean([df["completed_amount_"].iloc[i][nutrient] for i in best_match_indices]), 2)
            for nutrient in filtered_nutrients
        }
    else:
        avg_nutrient_dict = df["completed_amount_"].iloc[best_match_indices[0]]

    # Thêm các giá trị mới từ best match đầu tiên
    first_match_index = best_match_indices[0]
    food_portions_amount = df["foodPortions_amount"].iloc[first_match_index]
    gram_weight = df["gramWeight"].iloc[first_match_index]
    name_1 = df["name.1"].iloc[first_match_index]

    # if len(best_match_indices) > 10:
    #     results[name] = {
    #     "best_matches": None,
    #     "averaged_nutrients": None,
    #     "foodPortions_amount": None,
    #     "gramWeight": None,
    #     "name.1": None,
    # }
    results[name] = {
        "best_matches": [df["description"].iloc[i] for i in best_match_indices],
        "averaged_nutrients": avg_nutrient_dict,
        # "foodPortions_amount": food_portions_amount, # quy hết amount về 1 đơn vị
        "gramWeight": list(np.array(eval(gram_weight))/ np.array(eval(food_portions_amount))),
        # "gramWeight": gram_weight,
        "measureUnit": eval(name_1),
    }

results


{'almond': {'best_matches': ['Flour, almond'],
  'averaged_nutrients': {'Total lipid (fat)': 50.2,
   'Fiber, total dietary': 9.27,
   'Water': 4.32,
   'Protein': 26.2,
   'Carbohydrate, by difference': 16.2,
   'average_energy': 600.0,
   'Sugars, Total': 0.0},
  'gramWeight': [],
  'measureUnit': []},
 'apple': {'best_matches': ['Apples, fuji, with skin, raw',
   'Apples, gala, with skin, raw',
   'Apples, honeycrisp, with skin, raw'],
  'averaged_nutrients': {'average_energy': 58.8,
   'Carbohydrate, by difference': 15.07,
   'Water': 84.4,
   'Total lipid (fat)': 0.14,
   'Sugars, Total': 12.5,
   'Protein': 0.13,
   'Fiber, total dietary': 1.97},
  'gramWeight': [],
  'measureUnit': []},
 'asparagus': {'best_matches': ['Asparagus, green, raw'],
  'averaged_nutrients': {'Total lipid (fat)': 0.216,
   'Fiber, total dietary': 1.88,
   'Water': 92.6,
   'Protein': 1.44,
   'Carbohydrate, by difference': 5.1,
   'average_energy': 25.8,
   'Sugars, Total': 0.0},
  'gramWeight': [],
  '

In [45]:
np.array(eval(gram_weight))/ np.array(eval(food_portions_amount))

array([], dtype=float64)

In [2]:
import json
with open("ingredient_nutrient.json", "w") as outfile: 
    json.dump(results, outfile)

In [4]:
import json
with open("ingredient_nutrient.json", "r") as outfile: 
    data = json.load(outfile)

data

{'almond': {'best_matches': ['Flour, almond'],
  'averaged_nutrients': {'Total lipid (fat)': 50.2,
   'Fiber, total dietary': 9.27,
   'Water': 4.32,
   'Protein': 26.2,
   'Carbohydrate, by difference': 16.2,
   'average_energy': 600.0,
   'Sugars, Total': 0.0}},
 'apple': {'best_matches': ['Apples, fuji, with skin, raw',
   'Apples, gala, with skin, raw',
   'Apples, honeycrisp, with skin, raw'],
  'averaged_nutrients': {'average_energy': 58.800000000000004,
   'Carbohydrate, by difference': 15.066666666666668,
   'Water': 84.39999999999999,
   'Total lipid (fat)': 0.13733333333333334,
   'Sugars, Total': 12.5,
   'Protein': 0.12766666666666668,
   'Fiber, total dietary': 1.9699999999999998}},
 'asparagus': {'best_matches': ['Asparagus, green, raw'],
  'averaged_nutrients': {'Total lipid (fat)': 0.216,
   'Fiber, total dietary': 1.88,
   'Water': 92.6,
   'Protein': 1.44,
   'Carbohydrate, by difference': 5.1,
   'average_energy': 25.8,
   'Sugars, Total': 0.0}},
 'avocado': {'best_m

### Push data to database pgadmin4

In [ ]:
pip install psycopg2

In [ ]:
import json
import psycopg2

# Kết nối tới cơ sở dữ liệu PostgreSQL
conn = psycopg2.connect(
    dbname="database5", 
    user="postgres", 
    password="sehilnlf", 
    host="localhost", 
    port="5432"
)
cur = conn.cursor()

# Đọc file JSON
with open('ingredient_nutrient.json', 'r') as f:
    data = json.load(f)

# Chuẩn bị dữ liệu cho bảng 'ingredient'
for ingredient_name, nutrient_data in data.items():
    # Insert vào bảng 'ingredient'
    cur.execute("""
        INSERT INTO ingredient (ingredient_id, name, unit_name) 
        VALUES (DEFAULT, %s, %s) 
        RETURNING ingredient_id;
    """, (ingredient_name, 'g'))  # Giả sử đơn vị mặc định là 'g'

    ingredient_id = cur.fetchone()[0]

    for nutrient_name, amount in nutrient_data['averaged_nutrients'].items():
        # Kiểm tra nếu nutrient đã tồn tại
        cur.execute("""
            SELECT nutrient_id FROM nutrient WHERE name = %s;
        """, (nutrient_name,))
        result = cur.fetchone()
        
        if result:
            nutrient_id = result[0]
        else:
            # Chèn vào bảng 'nutrient' nếu chưa tồn tại
            cur.execute("""
                INSERT INTO nutrient (name, unit_name) 
                VALUES (%s, %s) 
                RETURNING nutrient_id;
            """, (nutrient_name, 'g'))  # Giả sử đơn vị mặc định là 'g'
            nutrient_id = cur.fetchone()[0]
        
        # Chèn vào bảng 'ingredient_contains_nutrient'
        cur.execute("""
            INSERT INTO ingredient_contains_nutrient (ingredient_id, nutrient_id, amount) 
            VALUES (%s, %s, %s);
        """, (ingredient_id, nutrient_id, amount))


# Commit các thay đổi và đóng kết nối
conn.commit()
cur.close()
conn.close()

In [6]:
# import bcrypt

# # Danh sách user_names
# user_names = ["john_doe", "jane_smith", "alice_w"]

# # Hàm để hash mật khẩu
# def hash_password(password):
#     # salt = bcrypt.gensalt()  # Tạo salt
#     hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt(10))

#     return hashed_password.decode('utf-8')  # Trả về chuỗi hash

# # Tạo danh sách hashed password cho các user
# hashed_passwords = [(user_name, hash_password(user_name)) for user_name in user_names]

# # In kết quả
# for user_name, hashed_pw in hashed_passwords:
#     print(f"User: {user_name}, Hashed Password: {hashed_pw}")




User: john_doe, Hashed Password: $2b$10$AfCB2bAsIuO2Onn9krDaG.3C874tPHnpbyhtpFSIwXtO3e61SABw.
User: jane_smith, Hashed Password: $2b$10$2tKSNzjMm3/q13lqehu2zOk7aWm/WSdRpqjDCQuIdvUpiXTE2H0nO
User: alice_w, Hashed Password: $2b$10$lsp7QkmE8tG0j0KrU/GXDuBv9MhAzLPQv7BMfeJ.V6cjH6KX6Qriu


In [ ]:
# -- Insert hashed passwords into 'users' table
# INSERT INTO users (user_name, gender, hashed_pw, current_weight, target_weight, timeframe)
# VALUES
#     ('john_doe', 'M', '$2b$12$6u5n9Rul2uHJkShTxU9TxeZyX5vRQpV85BOwDZ8zvcfxWoWgI4SuS', 75.0, 70.0, 12),
#     ('jane_smith', 'F', '$2b$12$L/Z1tS9jz1fOIF2H7Vt/feycbGQ5Y1WaJf5aJUmT1jF71b9ddGxOi', 60.0, 55.0, 10),
#     ('alice_w', 'F', '$2b$12$8fbBc4E3AzIo2zE5O3VWE.iBjaBOVN03ytCPSgVquI9mqsDdKJqD2', 68.5, 65.0, 8);


# -- Insert fake data into 'goal' table
# INSERT INTO goal (user_id, nutrient_id, amount)
# VALUES
#     (1, 1, 50.0),
#     (1, 2, 70.0),
#     (2, 1, 45.0),
#     (2, 3, 60.0),
#     (3, 2, 55.0),
#     (3, 3, 65.0);
